# Analyse scène compétitive Pokemon

## Analyse exploratoire

### Création des bases des données

Commençons par récupérer les données d'utilisation des différents pokemon sur la scène compétitive 6v6. Le site Smogon propose une banque de données conséquente sur ces utilisations au sein des différentes catégories compétitives.

Nous allons utiliser la classe d'objet *Contact_Smogon* pour récupérer les fichiers bruts aux formats *.txt* avant de les convertir en *.csv*. Dans un premier temps nous regarderons en premier lieu le format 7G OverUsed tous niveaux, nous allons donc récupérer toutes les données correspondantes, à raison d'un fichier par mois.

In [1]:
#On importe la classe requise, qui gère déjà les imports nécessaires
import contact_smogon as cs

#On initialise avant de lancer la requête, par défaut elle demandera toutes les dates disponibles
contact = cs.Contact_Smogon(gen='gen7',tier='ou',rating='0')
contact.request_data()

Folder /raw_data succesfully created
File /rawdata_2016-11_gen7ou-0.txt succesfully created
Folder /clean_data succesfully created
File /data_2016-11_gen7ou-0.txt.csv succesfully created
File /rawdata_2016-12_gen7ou-0.txt succesfully created
File /data_2016-12_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-02_gen7ou-0.txt succesfully created
File /data_2017-02_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-03_gen7ou-0.txt succesfully created
File /data_2017-03_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-04_gen7ou-0.txt succesfully created
File /data_2017-04_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-05_gen7ou-0.txt succesfully created
File /data_2017-05_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-06_gen7ou-0.txt succesfully created
File /data_2017-06_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-07_gen7ou-0.txt succesfully created
File /data_2017-07_gen7ou-0.txt.csv succesfully created
File /rawdata_2017-08_gen7ou-0.txt s

File /rawdata_2023-02_gen7ou-0.txt succesfully created
File /data_2023-02_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-03_gen7ou-0.txt succesfully created
File /data_2023-03_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-04_gen7ou-0.txt succesfully created
File /data_2023-04_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-05_gen7ou-0.txt succesfully created
File /data_2023-05_gen7ou-0.txt.csv succesfully created
File /rawdata_2023-06_gen7ou-0.txt succesfully created
File /data_2023-06_gen7ou-0.txt.csv succesfully created
